# Product Mix Optimization — Estudo de Caso
**Lua Smart Tech | Engenharia Química + Analytics**

Este notebook apresenta um estudo de caso de otimização de mix de produtos usando programação linear.
A decisão é **quantas unidades produzir de cada modelo** para maximizar o lucro, respeitando capacidade de montagem e teste.

**Resumo executivo**
- Modelos: Lua1 e Lua2 (smartphones)
- Objetivo: maximizar lucro mensal
- Restrições: horas de montagem e teste + limite de demanda
- Dados sintéticos para fins educacionais


## Contexto de negócio
A Lua Smart Tech fabrica dois modelos de smartphone. O time de operações precisa definir o plano de produção mensal
considerando custos, capacidade da fábrica e demanda esperada. O objetivo é entregar o maior lucro possível sem
ultrapassar limites de mão de obra.


## Decisão suportada
- Qual o mix ótimo de produção (Lua1 vs. Lua2)?
- Qual o lucro máximo esperado?
- Quais recursos são gargalo e onde existe folga?


## Premissas e dados
Premissas simplificadoras:
- Custos e tempos são lineares e constantes.
- Toda a produção é vendida até o limite de demanda.
- Não há estoque inicial nem backlog.

A tabela abaixo consolida os parâmetros por produto (valores em R$ e horas).


In [ ]:
import pandas as pd
from pulp import LpProblem, LpVariable, LpMaximize, LpInteger, lpSum, LpStatus, value


In [ ]:
products = ["Lua1", "Lua2"]

params = {
    "max_demand": {"Lua1": 600, "Lua2": 1200},
    "price": {"Lua1": 300, "Lua2": 450},
    "component_cost": {"Lua1": 150, "Lua2": 225},
    "assembly_hours": {"Lua1": 5, "Lua2": 6},
    "test_hours": {"Lua1": 1, "Lua2": 2},
    "assembly_cost_per_hour": 11,
    "test_cost_per_hour": 15,
    "max_assembly_hours": 10000,
    "max_test_hours": 3000,
}

profit_unit = {
    p: params["price"][p]
    - params["component_cost"][p]
    - params["assembly_hours"][p] * params["assembly_cost_per_hour"]
    - params["test_hours"][p] * params["test_cost_per_hour"]
    for p in products
}

df_params = pd.DataFrame(
    {
        "demanda_max": [params["max_demand"][p] for p in products],
        "preco_venda": [params["price"][p] for p in products],
        "custo_componentes": [params["component_cost"][p] for p in products],
        "horas_montagem": [params["assembly_hours"][p] for p in products],
        "horas_teste": [params["test_hours"][p] for p in products],
        "lucro_unitario": [profit_unit[p] for p in products],
    },
    index=products,
)

df_params


Capacidades:
- Montagem: 10.000 h
- Teste: 3.000 h


## Formulação do modelo (PL)
Variáveis de decisão:
- \(x_i\) = unidades do produto \(i\) a produzir no mês.

Função objetivo:
\[
\max \sum_i (B_i - C_i - D H_{f,i} - E H_{g,i}) x_i
\]

Restrições:
\[
\sum_i H_{f,i} x_i \le F \quad (\text{montagem})
\]
\[
\sum_i H_{g,i} x_i \le G \quad (\text{teste})
\]
\[
0 \le x_i \le A_i \quad \forall i
\]

Onde \(A_i\) é a demanda máxima, \(B_i\) o preço, \(C_i\) o custo de componentes,
\(D\) e \(E\) os custos de mão de obra por hora.


In [ ]:
model = LpProblem("product_mix", LpMaximize)

x = {p: LpVariable(f"units_{p}", lowBound=0, cat=LpInteger) for p in products}

model += lpSum(profit_unit[p] * x[p] for p in products)

model += (
    lpSum(params["assembly_hours"][p] * x[p] for p in products)
    <= params["max_assembly_hours"],
    "assembly_hours",
)
model += (
    lpSum(params["test_hours"][p] * x[p] for p in products)
    <= params["max_test_hours"],
    "test_hours",
)
for p in products:
    model += x[p] <= params["max_demand"][p], f"max_demand_{p}"


In [ ]:
model.solve()
LpStatus[model.status]


In [ ]:
results = pd.DataFrame(
    {
        "produto": products,
        "unidades": [x[p].value() for p in products],
        "lucro_unitario": [profit_unit[p] for p in products],
    }
)
results["lucro_total"] = results["unidades"] * results["lucro_unitario"]
results


In [ ]:
total_profit = value(model.objective)
total_profit


In [ ]:
assembly_used = sum(params["assembly_hours"][p] * x[p].value() for p in products)
test_used = sum(params["test_hours"][p] * x[p].value() for p in products)

util = pd.DataFrame(
    {
        "recurso": ["Montagem (h)", "Teste (h)"],
        "capacidade": [params["max_assembly_hours"], params["max_test_hours"]],
        "usado": [assembly_used, test_used],
    }
)
util["folga"] = util["capacidade"] - util["usado"]
util["utilizacao_%"] = (util["usado"] / util["capacidade"] * 100).round(1)
util


In [ ]:
demand_slack = pd.DataFrame(
    {
        "produto": products,
        "demanda_max": [params["max_demand"][p] for p in products],
        "produzido": [x[p].value() for p in products],
    }
)
demand_slack["folga_demanda"] = demand_slack["demanda_max"] - demand_slack["produzido"]
demand_slack


In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(1, 2, figsize=(10, 4))
ax[0].bar(results["produto"], results["unidades"], color=["#2E86AB", "#F18F01"])
ax[0].set_title("Mix de produção ótimo")
ax[0].set_ylabel("Unidades")

ax[1].bar(util["recurso"], util["utilizacao_%"], color=["#2E86AB", "#F18F01"])
ax[1].axhline(100, color="gray", linestyle="--", linewidth=1)
ax[1].set_title("Utilização de capacidade")
ax[1].set_ylabel("Utilização (%)")

plt.tight_layout()


## Conclusões e recomendações
- Plano ótimo: 560 unidades Lua1 e 1200 unidades Lua2.
- Lucro máximo estimado: R$ 199.600.
- Montagem é o gargalo (100%); teste possui folga de 40 h.
- Demanda de Lua2 é totalmente atendida; Lua1 fica 40 unidades abaixo do limite.
- Se o negócio puder ampliar capacidade, priorize horas de montagem.


## Próximos passos
- Análise de sensibilidade (demanda, preços, capacidade).
- Inserir custos fixos ou setup para aproximar a realidade industrial.
- Simular cenários com mais produtos.
